# 1학년 수강 과목을 통한 학부 예측 프로그램 만들기  
## 모두를 위한 인공지능 Final project - 11조 윤희원(21600481)/김유빈(21900167)  
### 프로젝트 소개  
수업시간에 배운 MNIST BGD 모델을 활용하여 한동대학교 18학번 학생들의 데이터를 학습시킨 뒤 실제 선택 학부를 얼마나 잘 예측하는 지 확인한다.  
  
### 프로젝트 목표  
1. 18학번들의 수강과목 데이터를 활용하여 2학년이 되었을때 어떤 학부를 선택할지 예측할 수 있다.  
2. MNIST 모델을 사용한 학습 모델의 구조를 이해할 수 있다.  
3. 18학번이 수강한 과목 데이터를 통해 어떤 학부 전공을 선택하게 될지 직접 인공지능 모델을 학습시킬 수 있다.  
  
### 프로젝트 개요
1. '모두를 위한 인공지능의 활용’ 수업 github에서 JoyML11-3BatchGD.ipynb 파일을 다운로드 한다.  
2. MNIST BGD 모델 코드를 활용하여 우리가 실제 학습하고자하는 신경망의 형태로 변환한다.  
3. 18학번들의 수강과목 데이터를 다운받아서 필요한 정보들만 추출하여 CSV 형태로 가공한다.  
4. 가공된 데이터를 Jupyter Notebook으로 읽어들인 후 학습시킨다.  
5. 테스트용 데이터를 활용하여 예측 정확도를 확인한다.  




## MNIST BGD 모델의 활용  
바로 아래의 셀은 MNIST BGD 모델에서 활용하는 함수들을 정의 해놓은 영역이다. 초기 가중치들의 설정, 순전파, 역전파 등의 알고리즘이 잘 정리되어 있다. 

In [1]:
class MnistBGD_LS(object):
    """ Batch Gradient Descent with Learning Schedule
    """
    def __init__(self, n_x, n_h1, n_y, eta = 0.1, epochs = 100, random_seed=1):
        """ 
        """
        self.n_x = n_x
        self.n_h = n_h
        self.n_y = n_y
        self.eta = eta
        self.epochs = epochs
        np.random.seed(random_seed)
        self.W1 = 2*np.random.random((self.n_h, self.n_x)) - 1  # between -1 and 1
        self.W2 = 2*np.random.random((self.n_y, self.n_h)) - 1  # between -1 and 1
        print('W1.shape={}, W2.shape={}'.format(self.W1.shape, self.W2.shape))
        
    def forpass(self, A0):
        Z1 = np.dot(self.W1, A0)        # hidden layer inputs
        A1 = self.g(Z1)                 # hidden layer outputs/activation func
        Z2 = np.dot(self.W2, A1)        # output layer inputs
        A2 = self.g(Z2)                 # output layer outputs/activation func
        return Z1, A1, Z2, A2

    def fit(self, X, y):
        self.cost_ = []
        self.m_samples = len(y)       
        Y = joy.one_hot_encoding(y, self.n_y)     
        # learning rate is scheduled to decrement by a step of 
        # which the inteveral from self.eta to 0.0001 eqaully 
        # divided by total number of iterations(epochs or 
        # epochs * m_samples)
        eta_scheduled = np.linspace(self.eta, 0.0001, self.epochs)
        
        # for momentum
        #self.v1 = np.zeros_like(self.W1)
        #self.v2 = np.zeros_like(self.W2)
        
        for epoch in range(self.epochs):
            if epoch % 1000 == 0:
                print('Training epoch {}/{}.'.format(epoch, self.epochs))

            A0 = np.array(X, ndmin=2).T       
            Y0 = np.array(Y, ndmin=2).T     

            Z1, A1, Z2, A2 = self.forpass(A0)  
            E2 = Y0 - A2                      
            E1 = np.dot(self.W2.T, E2)         

            dZ2 = E2 * self.g_prime(Z2)          
            dZ1 = E1 * self.g_prime(Z1)       
            
            # udpate weight with momentum
            #eta = learning_schedule[epoch]
            #self.v2 = 0.9 * self.v2 + self.eta * np.dot(dZ2, A1.T) / m_samples
            #self.v1 = 0.9 * self.v1 + self.eta * np.dot(dZ1, A0.T) / m_samples
            #self.W2 += self.v2     
            #self.W1 += self.v1 

            # update weights without momentum
            # eta = eta_scheduled[epoch]
            self.W2 +=  self.eta * np.dot(dZ2, A1.T) / self.m_samples    
            self.W1 +=  self.eta * np.dot(dZ1, A0.T) / self.m_samples    
            self.cost_.append(np.sqrt(np.sum(E2 * E2)))
        return self

    def predict(self, X):
        A0 = np.array(X, ndmin=2).T         # A0: inputs
        Z1, A1, Z2, A2 = self.forpass(A0)   # forpass
        return A2                                       

    def g(self, x):                 # activation_function: sigmoid
        x = np.clip(x, -500, 500)   # prevent from overflow, 
        return 1.0/(1.0+np.exp(-x)) # stackoverflow.com/questions/23128401/
                                    # overflow-error-in-neural-networks-implementation
    
    def g_prime(self, x):                    # activation_function: sigmoid derivative
        return self.g(x) * (1 - self.g(x))
    
    def evaluate(self, Xtest, ytest):       # fully vectorized calculation
        m_samples = len(ytest)
        scores = 0        
        A2 = self.predict(Xtest)
        yhat = np.argmax(A2, axis = 0)
        scores += np.sum(yhat == ytest)
        return scores/m_samples * 100
    
    def evaluate_onebyone(self, Xtest, ytest):
        m_samples = len(ytest)
        scores = 0
        for m in range(m_samples):
            A2 = nn.predict(Xtest[m])
            yhat = np.argmax(A2)
            if yhat == ytest[m]:
                scores += 1        
        return scores/m_samples * 100
    

## 가공된 데이터를 적용하여 학습시키기  
아래 셀에서는 CSV 파일의 형태로 가공된 학생들의 수강 과목과 선택 학부 데이터를 활용해 직접 학습을 시킨다.  
MNIST classification accuracy를 통해 자체적으로 학습된 데이터의 정확성에 대하여 알아볼 수 있다.  

In [2]:
import importlib
# import matplotlib
# import joy.py
# importlib.reload(joy)
# import csv
import numpy as np
# import matplotlib
# %matplotlib inline



## 새로운 데이터를 입력 받았을 경우  
새로운 학생들의 데이터를 입력 받을 때에 과연 어느 정도의 예측 정확성을 보여주기 위한 부분이다.  
이때, 현재 600명의 데이터만을 바탕으로 학습한 신경망으로는 20% 정도의 낮은 정확성만을 보여준다. 
비록 그렇다 할지라도 단순히 하나의 학부를 선택했을 경우인 1/12의 확률보다는 약 두 배 가량 높은 정확도를 보인다.  
따라서 만약 많은 양의 데이터만 확보된다면 이 신경망을 통해 보다 정확한 예측을 수행할 것으로 기대된다.

In [6]:
data3 = np.genfromtxt("class.csv", dtype=int, delimiter=',', names=True, encoding='UTF8')
#각 학생이 어떠한 과목을 수강하였는지를 숫자로 표현한 가공 데이터(class2)를 읽어들인다.
data4 = np.genfromtxt("major.csv", dtype=int, delimiter=',', names=True, encoding='UTF8')
#각 학생이 어떠한 학부를 선택하였는지를 숫자로 표현한 가공 데이터(major2)을 읽어들인다.

Xtest = np.zeros((34, 200), dtype = np.int32)
Ytest = np.zeros((34), dtype = np.int32)

for i in range(34):
    for j in range(22):
            if  data3[i][j] != 0:
                Xtest[i][data3[i][j]] = 1
                
for k in range(34): 
    Ytest[k] = data4[k][0]-1

accuracy = np.evaluate(Xtest, Ytest) 
print('MNIST classification accuracy {}%'.format(accuracy))

ValueError: Some errors were detected !
    Line #2 (got 16 columns instead of 17)
    Line #5 (got 18 columns instead of 17)
    Line #6 (got 18 columns instead of 17)
    Line #9 (got 20 columns instead of 17)
    Line #10 (got 19 columns instead of 17)
    Line #11 (got 18 columns instead of 17)
    Line #12 (got 15 columns instead of 17)
    Line #13 (got 18 columns instead of 17)
    Line #14 (got 21 columns instead of 17)
    Line #15 (got 18 columns instead of 17)
    Line #16 (got 19 columns instead of 17)
    Line #17 (got 16 columns instead of 17)
    Line #18 (got 19 columns instead of 17)
    Line #19 (got 19 columns instead of 17)
    Line #20 (got 19 columns instead of 17)
    Line #21 (got 18 columns instead of 17)
    Line #22 (got 19 columns instead of 17)
    Line #23 (got 18 columns instead of 17)
    Line #24 (got 19 columns instead of 17)
    Line #25 (got 18 columns instead of 17)
    Line #26 (got 14 columns instead of 17)
    Line #28 (got 18 columns instead of 